In [1]:
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper
from nuscenes.prediction.models.physics import ConstantVelocityHeading, PhysicsOracle
from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.prediction.input_representation.agents import AgentBoxesWithFadedHistory
from nuscenes.prediction.input_representation.interface import InputRepresentation
from nuscenes.prediction.input_representation.combinators import Rasterizer
from nuscenes.prediction.models.backbone import ResNetBackbone
from nuscenes.prediction.models.mtp import MTP, MTPLoss
from nuscenes.prediction.models.covernet import CoverNet, ConstantLatticeLoss

import torch
import numpy as np
from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.eval.prediction.data_classes import Prediction
import json
import os

import matplotlib.pyplot as plt
from typing import List, Dict, Any
from collections import defaultdict
import pickle

In [3]:
os.path.join(os.getcwd(), 'maps')

'/home/armahade/ECE285/AV/final_proj/MTP_experiments/maps'

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 32.1 seconds.
Reverse indexing ...
Done reverse indexing in 8.3 seconds.


In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader


from nuscenes import NuScenes

from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper

from nuscenes.prediction.models.physics import ConstantVelocityHeading, PhysicsOracle
from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.prediction.input_representation.agents import AgentBoxesWithFadedHistory
from nuscenes.prediction.input_representation.interface import InputRepresentation
from nuscenes.prediction.input_representation.combinators import Rasterizer
from PIL import Image



class NuscenesDataset(Dataset):
    
    def __init__(self, 
                 nusc,
                 helper,
                 maps_dir,
                 save_maps_dataset = False, 
                 config_name = 'predict_2020_icra.json',
                 history=1, 
                 in_agent_frame=True):
        
        
        self.nusc = nusc
        self.helper = helper
        
        #initialize the data set 
        if maps_dir == 'maps_train':
            dataset_version = "train"
        elif maps_dir == 'maps_val':
            dataset_version = "val"
        
        #initialize maps directory where everything will be saved 
        self.maps_dir = os.path.join(os.getcwd(), maps_dir)

        self.data_set = get_prediction_challenge_split(dataset_version,dataroot=self.nusc.dataroot)

        #initialize rasterizers for map generation
        self.static_layer_rasterizer = StaticLayerRasterizer(self.helper)
        self.agent_rasterizer = AgentBoxesWithFadedHistory(self.helper, seconds_of_history=history)
        self.mtp_input_representation = InputRepresentation(self.static_layer_rasterizer, self.agent_rasterizer, Rasterizer())

        self.in_agent_frame = in_agent_frame

        self.config = load_prediction_config(self.helper, config_name)

        self.valid_data_points = []
        
        self.save_maps_dataset = save_maps_dataset
        
        if self.save_maps_dataset: 
            self.save_maps()
            
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

            
    def save_maps(self):
        '''
        Input: None 
        Output: None
        
        This method finds all the valid data points in the data set. We define a valid data point 
        where the velocity, acceleartion, and heading specified by token not NaN. 
        '''
        print("starting to save maps")
        for i, token in enumerate(self.data_set): 
            instance_token_img, sample_token_img = self.data_set[i].split('_')
            
            file_path = os.path.join(self.maps_dir, "maps_{0}.jpg".format(i))
            
            instance_token_img, sample_token_img = self.data_set[i].split('_')
            img = self.mtp_input_representation.make_input_representation(instance_token_img, sample_token_img)
            im = Image.fromarray(img)
            im.save(file_path)
        
            print("{0}/{1} image saved".format(i, len(self.data_set)))
        
        print("done saving maps")
        
        
        
    def __len__(self):
        return len(self.data_set)
    
    #return the image tensor, agent state vector, and the ground truth
    def __getitem__(self, index):
        
        instance_token_img, sample_token_img = self.data_set[index].split('_')
        
        velocity = self.helper.get_velocity_for_agent(instance_token_img, sample_token_img)
        acceleration = self.helper.get_acceleration_for_agent(instance_token_img, sample_token_img)
        heading = self.helper.get_heading_change_rate_for_agent(instance_token_img, sample_token_img)        

        #using a padding token of -1
        if np.isnan(velocity) or np.isnan(acceleration) or np.isnan(heading):
            velocity =  acceleration = heading = -1 

        #construct agent state vector 
        agent_state_vec = torch.Tensor([velocity, acceleration, heading])
        
        #change image from (3, N, N), will have data loader take care 
        #get image and construct tensor 
        file_path = os.path.join(self.maps_dir, "maps_{0}.jpg".format(index))
        
        im = Image.open(file_path)
        img = np.array(im)
        image_tensor = torch.Tensor(img).permute(2, 0, 1)
        
        #get ground truth 
        ground_truth = self.helper.get_future_for_agent(instance_token_img, 
                                                        sample_token_img,
                                                        self.config.seconds, 
                                                        in_agent_frame=self.in_agent_frame)
        
        ground_truth = torch.Tensor(ground_truth).unsqueeze(0)
        return image_tensor, agent_state_vec, ground_truth 

In [ ]:
import torch.optim as optim
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataloader = DataLoader(dataset, batch_size=8, shuffle = True, pin_memory = True, num_workers = 4)

#init network 
count = 0 
M = 20
backbone = ResNetBackbone('resnet50')
mtp = MTP(backbone, num_modes=M).to(device)

# covernet = CoverNet(backbone, num_modes=64).to(device)
#init loss function and optimizers
# criterion = ConstantLatticeLoss(trajectories)

criterion = MTPLoss(num_modes=M)
optimizer = optim.Adam(mtp.parameters(), lr=3e-4)

output_dir = '.'
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)
losses = []
print('starting 100 epochs')
print('total batches:' + str(len(dataloader)))
for i in range(1):
    epoch_loss = 0 
    count = 0 
    for image_tensor, agent_vec, ground_truth in dataloader:
        output = mtp(image_tensor.to(device), agent_vec.to(device))
        loss = criterion(output, ground_truth.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count+=1
        epoch_loss+=loss
        print("{0}/{1} loss is: {2}".format(count + 1, len(dataloader), loss))
        
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

In [ ]:
!python train.py

In [ ]:
#import map rasterizer for agents 

In [4]:
#import config from
from nuscenes.eval.prediction.config import PredictionConfig, load_prediction_config
baseDir = '../full_data/sets/nuscenes'
nusc = NuScenes(version='v1.0-trainval', dataroot=baseDir, verbose=True)
helper = PredictHelper(nusc)
val = get_prediction_challenge_split("train", dataroot='../full_data/sets/nuscenes')
config_name = 'predict_2020_icra.json'
config = load_prediction_config(helper, config_name)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 39.9 seconds.
Reverse indexing ...
Done reverse indexing in 17.9 seconds.
static_layers.py - Loading Map: singapore-queenstown
static_layers.py - Loading Map: singapore-hollandvillage
static_layers.py - Loading Map: boston-seaport
static_layers.py - Loading Map: singapore-onenorth


In [ ]:
print(len(mtp_output))

In [6]:


def compute_metrics(predictions: List[Dict[str, Any]],
                    helper: PredictHelper, config: PredictionConfig) -> Dict[str, Any]:
    """
    Computes metrics from a set of predictions.
    :param predictions: List of prediction JSON objects.
    :param helper: Instance of PredictHelper that wraps the nuScenes val set.
    :param config: Config file.
    :return: Metrics. Nested dictionary where keys are metric names and value is a dictionary
        mapping the Aggregator name to the results.
    """
    n_preds = len(predictions)
    containers = {metric.name: np.zeros((n_preds, metric.shape)) for metric in config.metrics}
    for i, prediction_str in enumerate(predictions):
        prediction = Prediction.deserialize(prediction_str)
        ground_truth = helper.get_future_for_agent(prediction.instance, prediction.sample,
                                                   config.seconds, in_agent_frame=True)
        for metric in config.metrics:
            containers[metric.name][i] = metric(ground_truth, prediction)
    aggregations: Dict[str, Dict[str, List[float]]] = defaultdict(dict)
    for metric in config.metrics:
        for agg in metric.aggregators:
            aggregations[metric.name][agg.name] = agg(containers[metric.name])
    return aggregations 


In [7]:
predictions = json.load(open('mtp_preds.json', "r"))
results = compute_metrics(predictions, helper, config)

In [8]:
results

defaultdict(dict,
            {'MinFDEK': {'RowMean': [20.517854563541597,
               8.908261344616186,
               8.908261344616186]},
             'MinADEK': {'RowMean': [8.246023765371822,
               4.338118872429088,
               4.338118872429088]},
             'MissRateTopK_2': {'RowMean': [0.9704678685986063,
               0.9164915385466209,
               0.9164915385466209]},
             'OffRoadRate': {'RowMean': [1.0]}})

In [ ]:
image_tensor = torch.Tensor(img).permute(2, 0, 1).unsqueeze(0)

print(agent_state_vector)
print(img.shape)
print(image_tensor.shape)

In [ ]:
# Output has 50 entries.
# The first 24 are x,y coordinates (in the agent frame) over the next 6 seconds at 2 Hz for the first mode.
# The second 24 are the x,y coordinates for the second mode.
# The last 2 are the logits of the mode probabilities
output = mtp(image_tensor, agent_state_vector)

In [ ]:
print(output.shape)

In [ ]:
logits = output[:,-M:]
print(logits)
probs = torch.softmax(logits, dim = 1)
best_mode = torch.argmax(probs,dim=1).item()
print(best_mode)

In [ ]:
future_xy_local = helper.get_future_for_agent(instance_token_img, sample_token_img, seconds=6, in_agent_frame=False)
future_xy_local

In [ ]:
output

In [ ]:
future_xy_local.shape

In [ ]:
future_xy_vec = future_xy_local.reshape(1, 24)
future_xy_vec

In [ ]:
output.shape
output[:,:-1].shapefu

In [ ]:
future_xy_vec - output[:,:-1].detach().numpy()

In [ ]:
output[:,:-1]

In [ ]:
future_xy_vec